In [2]:
import tensorflow_datasets as tfds

In [3]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

2024-06-13 22:31:01.644495: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 22:31:01.678871: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 22:31:01.678917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 22:31:01.678944: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 22:31:01.685641: I tensorflow/core/platform/cpu_feature_g

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /home/polivares/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


2024-06-13 22:31:12.682912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-13 22:31:12.683190: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-13 22:31:12.724011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
test_set, valid_set, train_set = tfds.load("tf_flowers",split=["train[0%:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True )

In [5]:
import tensorflow as tf

In [6]:
# Previo a la utilización de nuestro dataset en la red neuronal, la red que utilizaremos provee de
# funcionalidades de pre-procesamiento ya incluídas. Por lo tanto, utilizaremos estas funcionalidades
# y las dejaremos dentro de una función
def preprocess(image, label):
    # Cambiaremos las dimensiones de la imagen de entrada
    resized_image = tf.image.resize(image, [224, 224]) # Guardamos la imagen con nuevas dimensiones 224x224
    # Luego pasamos la imagen modificada en tamaño al preprocesamiento de nuestra red. La red
    # que utilizaremos de ejemplo tiene por nombre Xception
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    # Finalmente se retorna una imagen pre procesada (según lo indique el preprocess de xception)
    # y su etiqueta
    return final_image, label

In [7]:
# En este apartado hacemos los batch de datos directamente desde el dataset cargado por tensorflow
batch_size = 32

# Mezclamos el dataset 
train_set = train_set.shuffle(1000)
# Tanto para training, test y validación, aplicamos la función de preprocesamiento (preprocess)
# y luego generamos los batch de datos
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
# Acá empezamos con transfer learning. Lo que haremos será cargar una arquitectura de red neuronal desde
# tensorflow ya entrenada. Eso quiere decir que, no solo estamos cargando la arquitectura, con sus neuronas
# y conexiones, sino que también estamos cargando los PESOS DE ENTRENAMIENTO.

# Weights indica si utilizaremos pesos pre entrenados con el dataset imagenet o no
# include_top es el parámetro que indica explícitamente si quieres o no la capa de salida original
# de esta red.
# En base_model tenemos cargado nuestro modelo Xception sin la capa de salida. Nosotros podemos
# poner NUESTRA PROPIA CAPA(S) DE SALIDA
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
# Acá agregamos nuestras capas adicionales
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5, activation="softmax")(avg)

# Podemos conectar el input de nuestro modelo base con el output recién generado a través de 
# un modelo de Keras
model = tf.keras.Model(inputs=base_model.input, outputs=output)
# En este punto tenemos un nuevo modelo que utiliza como base la arquitectura Xception

In [9]:
# También nosotros conversamos que es posible decidir si queremos reentrenar aquellas capas ya entrenadas

# Acá vamos capa por capa modificando el parámetro "trainable" que permite (o no) reentrenar dicha capa
for layer in base_model.layers:
    layer.trainable = False # Esto impide que las capas se re entrenen

# Con este for, sobre todas las capas de nuestro modelo, estamos impidiendo que se reentrene
# alguna de sus capas ya entrenadas

In [10]:
# Proceso de compilación  (tal cual vimos en las clases anteriores)
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", 
              metrics=["accuracy"])

In [11]:
history = model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10


2024-06-13 22:38:21.427747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8906


 3/86 [>.............................] - ETA: 3s - loss: 1.6249 - accuracy: 0.1771  

2024-06-13 22:38:22.209393: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x74f334042b40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-13 22:38:22.209415: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-06-13 22:38:22.209418: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-06-13 22:38:22.268101: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


86/86 [==============================] - 8s 57ms/step - loss: 0.9829 - accuracy: 0.6831 - val_loss: 0.7273 - val_accuracy: 0.7641
Epoch 2/10
86/86 [==============================] - 4s 47ms/step - loss: 0.6083 - accuracy: 0.8180 - val_loss: 0.5988 - val_accuracy: 0.7985
Epoch 3/10
86/86 [==============================] - 4s 47ms/step - loss: 0.5102 - accuracy: 0.8430 - val_loss: 0.5424 - val_accuracy: 0.8094
Epoch 4/10
86/86 [==============================] - 4s 47ms/step - loss: 0.4580 - accuracy: 0.8601 - val_loss: 0.5080 - val_accuracy: 0.8167
Epoch 5/10
86/86 [==============================] - 4s 46ms/step - loss: 0.4237 - accuracy: 0.8674 - val_loss: 0.4867 - val_accuracy: 0.8385
Epoch 6/10
86/86 [==============================] - 4s 47ms/step - loss: 0.3985 - accuracy: 0.8765 - val_loss: 0.4677 - val_accuracy: 0.8348
Epoch 7/10
86/86 [==============================] - 4s 46ms/step - loss: 0.3782 - accuracy: 0.8801 - val_loss: 0.4530 - val_accuracy: 0.8494
Epoch 8/10
86/86 [======

In [12]:
model.evaluate(test_set)

12/12 [==============================] - 1s 51ms/step - loss: 0.3949 - accuracy: 0.8638


[0.39485207200050354, 0.863760232925415]

In [ ]:
for image, label in test_set:
    print("Etiqueta real",label)
    print("Predicción", model.predict(image))
